In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os

import tws_data 
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from ib_insync import *
import numpy as np
import time
import pandas as pd
import datetime as dt
pd.options.display.float_format = '{:,.4f}'.format
%matplotlib inline
from scipy.stats import norm as norm
import matplotlib.pyplot as plt
import requests as req
from bs4 import BeautifulSoup as bs
import math

#from sqlalchemy import *
from sqlalchemy import create_engine

def time_block_strtoactual(i):
    curr_time_block = dt.datetime.today().replace(hour = 9, minute = 30, second = 0, microsecond = 0) + dt.timedelta(minutes = 30*(i-1))
    return curr_time_block.strftime("%H:%M")

os.chdir('D:\Price Data\SPX and VIX Intraday TWS')

ib_index_engine = create_engine('sqlite:///ib_index_intraday.db', echo = False)

In [ ]:
current_end_date = dt.datetime(2013,1,31)

end_date = '{} 13:00:00 PST'.format(current_end_date.strftime('%Y%m%d'))
end_date

In [ ]:
vix_bars = range(2)

while len(vix_bars) > 0:
    
    client_id = 19

    ib = IB()
    ib.connect('127.0.0.1', 7496, clientId=client_id)

    spx = Index('SPX', 'CBOE')
    vix = Index('VIX', 'CBOE')
    
    ib.qualifyContracts(spx)
    ib.qualifyContracts(vix)
    
    ib.reqHeadTimeStamp(spx, whatToShow='TRADES', useRTH=True)
    ib.reqHeadTimeStamp(vix, whatToShow='TRADES', useRTH=True)

    duration = '10 D'

    spx_bars = ib.reqHistoricalData(
            spx,
            endDateTime=end_date,
            durationStr=duration,
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1)

    vix_bars = ib.reqHistoricalData(
            vix,
            endDateTime=end_date,
            durationStr=duration,
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1)

    if len(vix_bars) > 0:
        spx_df = util.df(spx_bars).set_index('date')
        spx_df.index = pd.to_datetime(spx_df.index)

        vix_df = util.df(vix_bars).set_index('date')
        vix_df.index = pd.to_datetime(vix_df.index)

        curr_df = spx_df.join(vix_df, lsuffix = '_spx', rsuffix = '_vix').dropna()

        ib.disconnect()
        curr_df.to_sql('indexIntraday', con=ib_index_engine, if_exists='append')
        current_end_date = curr_df.index[0] - dt.timedelta(days = 1)
        end_date = '{} 13:00:00 PST'.format(current_end_date.strftime('%Y%m%d'))
        print(end_date)
    else:
        ib.disconnect()
        break
    
    time.sleep(90)
